In [1]:
from datetime import datetime
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import requests



# Parameters
locationName = "Tokyo"
locationCountry = "JP"

url = "http://localhost:4000/historical/pastHistoricalData"

response = requests.post(url, json={"location": {"name": locationName, "country": locationCountry}}, timeout=120)
if response.status_code == 200:
    historical = response.json()
else:
    historical = []


if historical:
    data = pd.DataFrame(historical)
    data['date'] = pd.to_datetime(data['date'])
else:
    data = pd.DataFrame()

features = ['temperature', 'minTemperature', 'maxTemperature', 'precipitation', 'pressure']
for col in features:
    if col not in data.columns:
        data[col] = 0

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[features])

def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(x), np.array(y)

seq_length = 30
x, y = create_sequences(scaled_data, seq_length)

x = x.reshape((x.shape[0], x.shape[1], len(features)))

model = Sequential()
model.add(Input(shape=(seq_length, len(features))))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(features)))
model.compile(optimizer='adam', loss='mse')

model.fit(x, y, epochs=50, batch_size=16, verbose=1)

last_seq = scaled_data[-seq_length:]
forecast = []
input_seq = last_seq.copy()

for i in range(7):
    pred = model.predict(input_seq.reshape(1, seq_length, len(features)))
    forecast.append(pred[0])
    input_seq = np.vstack([input_seq[1:], pred])

forecast = scaler.inverse_transform(np.array(forecast))
forecast = forecast + np.random.normal(0, 1, size=forecast.shape)  # mean=0, std=1

date = pd.Timestamp.today()
forecast_dates = pd.date_range(start=date + pd.Timedelta(days=1), periods=7)
forecast_data = [
    {
        "date": str(d.date()),
        "temperature": float(f[0]),
        "minTemperature": float(f[1]),
        "maxTemperature": float(f[2]),
        "precipitation": float(f[3]),
        "pressure": float(f[4]),
    }
    for d, f in zip(forecast_dates, forecast)
]

forecast_data

Epoch 1/50


 1/20 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 0.2185

 9/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1842

15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1573

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.1391


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


[{'date': '2025-05-11',
  'temperature': 19.127314713373092,
  'minTemperature': 16.322812299484646,
  'maxTemperature': 21.86343843873064,
  'precipitation': 6.299118442905123,
  'pressure': 1015.0882432766389},
 {'date': '2025-05-12',
  'temperature': 18.42285513752471,
  'minTemperature': 14.06481824054069,
  'maxTemperature': 22.899075902819853,
  'precipitation': 6.30440464234827,
  'pressure': 1015.347341665135},
 {'date': '2025-05-13',
  'temperature': 17.85114135763666,
  'minTemperature': 14.176143235779927,
  'maxTemperature': 21.597853984907083,
  'precipitation': 5.074196524070177,
  'pressure': 1015.1261549278911},
 {'date': '2025-05-14',
  'temperature': 18.58879577766806,
  'minTemperature': 14.19815998695069,
  'maxTemperature': 23.84290229455645,
  'precipitation': 4.861203737445694,
  'pressure': 1016.0614249316595},
 {'date': '2025-05-15',
  'temperature': 18.701660579708552,
  'minTemperature': 14.865735196090116,
  'maxTemperature': 21.36528858147481,
  'precipitat

In [2]:
# Parameters
locationName = "Budapest / Ferihegy"
locationCountry = "HU"
